In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install nnunetv2

Mounted at /content/drive


In [ ]:
# Import basic packages for later use

import os
import shutil
from collections import OrderedDict, defaultdict
import re

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

import nnunetv2
import zipfile

In [1]:
!unrar x "/content/drive/MyDrive/nnUNET_SELs.rar" "/content/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Cannot open /content/drive/MyDrive/nnUNET_SELs.rar
No such file or directory
No files to extract


#Tree Folder Structure

In [ ]:
import os
import shutil
from collections import defaultdict

def setup_nnunet_dir(base_directory, dataset_name):
    """
    Sets up nnUNet directories and organizes files based on their names.

    Args:
    - base_directory (str): The base directory where nnUNet directories will be created.
    - dataset_name (str): The name of the dataset, e.g., "Dataset101".
    """
    # Create nnUNet directories
    nnunet_directory = os.path.join(base_directory, "nnUNet")
    os.makedirs(nnunet_directory, exist_ok=True)

    # Create nnUNet_preprocessed and nnUNet_results directories inside nnUNet
    for subdirectory in ["nnUNet_preprocessed", "nnUNet_results"]:
        os.makedirs(os.path.join(nnunet_directory, subdirectory), exist_ok=True)

    # Create directories nnUNet_raw/Dataset101_SEL/imagesTr and nnUNet_raw/Dataset101/labelsTr
    for subdirectory in ["imagesTr", "imagesTs", "labelsTr"]:
        os.makedirs(os.path.join(nnunet_directory, "nnUNet_raw", dataset_name, subdirectory), exist_ok=True)

    # Directory where the images are located
    images_directory = os.path.join(base_directory, "nnUNet", "nnUNet_raw", dataset_name)

    # Dictionary to store files by group (based on the first number between "_XXX_")
    files_by_group = defaultdict(list)

    # Iterate over the files in the directory
    for file in os.listdir(base_directory):
        if file.endswith(".nii.gz"):
            # Check if the file contains the word "SEL" or "lesion_mask"
            if "SEL" in file:
                os.remove(os.path.join(base_directory, file))
            elif "lesion_mask" in file:
                shutil.move(os.path.join(base_directory, file), os.path.join(nnunet_directory, "nnUNet_raw", dataset_name, "labelsTr", file))
            else:
                shutil.move(os.path.join(base_directory, file), os.path.join(nnunet_directory, "nnUNet_raw", dataset_name, "imagesTr", file))

# Example usage:
base_directory = "/content/nnUNet"
dataset_name = input("Enter the name of the dataset, e.g., 'Dataset101': ")
setup_nnunet_dir(base_directory, dataset_name)



## imagesTr structure

In [ ]:
import os
import re

def rename_files(directory):
    # Regular expression to find a three-digit number between underscores in the filename
    pattern = re.compile(r"_(\d{3})_")

    # Dictionary to keep track of the final number for each unique three-digit number
    final_numbers = {}

    for filename in os.listdir(directory):
        if filename.endswith(".nii.gz"):
            # Match with the pattern
            match = pattern.search(filename)

            if match:
                # Extract the three-digit number
                number = match.group(1)

                # Increment the final number for this three-digit number
                if number not in final_numbers:
                    final_numbers[number] = 0
                else:
                    final_numbers[number] += 1

                # New filename
                new_filename = f"{number}_{final_numbers[number]:04d}.nii.gz"

                os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

# Example usage:
directory = "/content/nnUNet/nnUNet_raw/Dataset101/imagesTr"
rename_files(directory)

In [ ]:
def get_basal(directory):

    for filename in os.listdir(directory):
        if filename.endswith(".nii.gz") and not (filename.endswith("0.nii.gz") or filename.endswith("1.nii.gz")):
            os.remove(os.path.join(directory, filename))

# Example usage:
directory = "/content/nnUNet/nnUNet_raw/Dataset101/imagesTr"  # Adjust the directory path as needed
get_basal(directory)

## labelTr structure

In [ ]:
def rename_labelTr(directory):

    # Regular expression to find a three-digit number in the filename
    pattern = re.compile(r"(\d{3})")

    for filename in os.listdir(directory):
        if filename.endswith(".nii.gz"):
            # Match with the pattern
            match = pattern.search(filename)
            if match:
                # New filename
                new_filename = f"{match.group(1)}.nii.gz"
                os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

# Example usage:
directory = "/content/nnUNet/nnUNet_raw/Dataset101/labelsTr"  # Adjust the directory path as needed
rename_labelTr(directory)

## Train Test Split

In [ ]:
import os
import random
import shutil
import re

def move_to_imagesTs(imagesTr_directory):

    # Create directory for imagesTs
    imagesTs_directory = os.path.join(imagesTr_directory, "..", "imagesTs")
    os.makedirs(imagesTs_directory, exist_ok=True)

    # Get all filenames in the imagesTr directory
    filenames = [f for f in os.listdir(imagesTr_directory) if f.endswith(".nii.gz")]

    # Extract identifiers (XXX) from filenames
    pattern = re.compile(r"(\d{3})_000[01]\.nii.gz")
    identifiers = set(match.group(1) for filename in filenames if (match := pattern.match(filename)))

    # Calculate the number of identifiers to move to imagesTs (20%)
    num_identifiers_to_move = int(len(identifiers) * 0.2)
    identifiers_list = list(identifiers)

    # Randomly select identifiers to move
    identifiers_to_move = random.sample(identifiers_list, num_identifiers_to_move)

    # Move the selected identifiers' files to imagesTs
    for identifier in identifiers_to_move:
        files_with_identifier = [f for f in filenames if f.startswith(f"{identifier}_")]
        files_to_move = random.sample(files_with_identifier, min(2, len(files_with_identifier)))  # Ensure we have at most 2 files per identifier
        for filename in files_to_move:
            shutil.move(os.path.join(imagesTr_directory, filename), os.path.join(imagesTs_directory, filename))

    return list(identifiers_to_move)

# Example usage:
imagesTr_directory = "/content/nnUNet/nnUNet_raw/Dataset101/imagesTr"
identifiers_imagesTs = move_to_imagesTs(imagesTr_directory)
print("Identifiers extracted from filenames:", identifiers_imagesTs)

In [ ]:
def move_to_labelsTest(dataset_directory, identifiers):
    """
    Moves files from labelsTr to labelsTest if their identifier matches any identifier in the list.

    Args:
    - dataset_directory (str): The directory path containing the dataset folders (labelsTr, labelsTest, etc.).
    - identifiers (list): List of identifiers to check against.
    """
    # Create directory for labelsTest
    labelsTest_directory = os.path.join(dataset_directory, "labelsTest")
    os.makedirs(labelsTest_directory, exist_ok=True)

    # Iterate over identifiers
    for identifier in identifiers:
        # Check if any file in labelsTr contains the identifier
        for filename in os.listdir(os.path.join(dataset_directory, "labelsTr")):
            if identifier in filename:
                # Move file to labelsTest
                shutil.move(os.path.join(dataset_directory, "labelsTr", filename), os.path.join(labelsTest_directory, filename))

# Example usage:
dataset_directory = "/content/nnUNet/nnUNet_raw/Dataset101"
identifiers = identifiers_imagesTs
move_to_labelsTest(dataset_directory, identifiers)

In [ ]:
### NO ES FA SERVIR

# Directorio donde se encuentran las imágenes
directorio = "/content/nnUNet/nnUNet_raw/Dataset101/labelsTr"
directorio_imagenes = os.listdir(directorio)
regex = r'_(\d+)_'

def ordenar_por_numeros(nombre_archivo):
    # Separar el nombre del archivo en partes usando "_"
    partes = nombre_archivo.split('_')
    # Extraer el número entre los guiones bajos
    numero = int(partes[1])
    return numero

directorio_imagenes = sorted(directorio_imagenes, key=ordenar_por_numeros)

for i in range(len(directorio_imagenes)):

    if i % 3 == 0:

        id_1 = re.findall(regex, directorio_imagenes[i])[0]
        id_2 = re.findall(regex, directorio_imagenes[i + 1])[0]
        id_3 = re.findall(regex, directorio_imagenes[i + 2])[0]


        if id_1 == id_2 == id_3:


            ruta_imagen1 = os.path.join(directorio, "rMSVIS_" + id_1 + "_T1_definite_SELs_mask.nii.gz")
            ruta_imagen2 = os.path.join(directorio, "rMSVIS_" + id_1 + "_T1_non_SELs_mask.nii.gz")
            ruta_imagen3 = os.path.join(directorio, "rMSVIS_" + id_1 + "_T1_possible_SELs_mask.nii.gz")

            # Cargar las imágenes MRI
            imagen1 = nib.load(ruta_imagen1)
            imagen2 = nib.load(ruta_imagen2)
            imagen3 = nib.load(ruta_imagen3)

            # Obtener los datos de las imágenes en un array numpy de 3 dimensiones
            datos_imagen1 = imagen1.get_fdata()
            datos_imagen2 = imagen2.get_fdata()
            datos_imagen3 = imagen3.get_fdata()

            # Aplicar umbralización para convertir los datos a valores binarios (0 o 1)
            datos_imagen1_binario = (datos_imagen1 > 0).astype(np.float32)
            datos_imagen2_binario = (datos_imagen2 > 0).astype(np.float32)
            datos_imagen3_binario = (datos_imagen3 > 0).astype(np.float32)

            # Aplicar el mapeo a los datos de la máscara SEL
            datos_imagen1_binario_mapped = np.where(datos_imagen1_binario == 1, 2, 0)
            datos_imagen3_binario_mapped = np.where(datos_imagen3_binario == 1, 2, 0)

            # Combinar las dos imágenes en una sola
            datos_imagen_combinada = datos_imagen1_binario_mapped + datos_imagen3_binario_mapped + datos_imagen2_binario
            datos_imagen_combinada[datos_imagen_combinada == 3] = 2
            datos_imagen_combinada[np.isnan(datos_imagen_combinada)] = 0

            # Crear una instancia de Nifti1Image con los datos de la máscara combinada
            output_nii_combinada = nib.Nifti1Image(datos_imagen_combinada, affine=imagen1.affine)

            # Imprimir los labels únicos en la máscara combinada
            unique_labels_combinada = np.unique(output_nii_combinada.get_fdata())
            print("Unique labels in the combined image:", unique_labels_combinada, id_1 )

            # Guardar la nueva imagen en formato NIfTI (.nii.gz)
            nombre_archivo_combinado = id_1 + ".nii.gz"
            ruta_completa_archivo_combinado = os.path.join(directorio, nombre_archivo_combinado)
            nib.save(output_nii_combinada, ruta_completa_archivo_combinado)

## Set Env var

In [ ]:
output_folder = '/content/nnUNet/nnUNet_raw/Dataset101'
channel_names = {0: 'BaseLineFL', 1: 'BaseLineT1'}
labels = {'background':0, 'lesion': 1}
file_ending = '.nii.gz'
num_training_cases = 106
dataset_name = 'sels'
description = 'sels segmentation'

In [ ]:
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
generate_dataset_json(output_folder=output_folder,channel_names=channel_names, labels=labels,
                      file_ending=file_ending,
                      num_training_cases=num_training_cases, dataset_name=dataset_name, description=description)

In [ ]:
os.environ['nnUNet_raw'] = "/content/nnUNet/nnUNet_raw"
os.environ['nnUNet_preprocessed'] =  "/content/nnUNet/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/nnUNet/nnUNet_results"

# Verify dataset integrity has to be executed only the first time you are pre-processing the data
# After successful plan and preprocessing
!nnUNetv2_plan_and_preprocess -d 101 --verify_dataset_integrity

In [ ]:
!nnUNetv2_train Dataset101_SEL 3d_fullres 0 -tr nnUNetTrainer_10epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-05-04 08:01:04.374875: do_dummy_2d_data_aug: False
2024-05-04 08:01:04.375678: Using splits from existing split file: /content/nnUNET_SELs/nnUNet_SEL/nnUNet_preprocessed/Dataset101_SEL/splits_final.json
2024-05-04 08:01:04.375939: The split file contains 5 splits.
2024-05-04 08:01:04.376014

In [ ]:
!nnUNetv2_train Dataset101_SEL 3d_fullres 1 -tr nnUNetTrainer_10epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-05-04 08:29:35.469040: do_dummy_2d_data_aug: False
2024-05-04 08:29:35.469932: Using splits from existing split file: /content/nnUNET_SELs/nnUNet_SEL/nnUNet_preprocessed/Dataset101_SEL/splits_final.json
2024-05-04 08:29:35.470188: The split file contains 5 splits.
2024-05-04 08:29:35.470253

In [ ]:
!nnUNetv2_train Dataset101_SEL 3d_fullres 2 -tr nnUNetTrainer_10epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-05-04 08:57:30.432538: do_dummy_2d_data_aug: False
2024-05-04 08:57:30.433453: Using splits from existing split file: /content/nnUNET_SELs/nnUNet_SEL/nnUNet_preprocessed/Dataset101_SEL/splits_final.json
2024-05-04 08:57:30.433709: The split file contains 5 splits.
2024-05-04 08:57:30.433778

In [ ]:
!nnUNetv2_train Dataset101_SEL 3d_fullres 3 -tr nnUNetTrainer_10epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-05-04 09:25:02.326064: do_dummy_2d_data_aug: False
2024-05-04 09:25:02.326958: Using splits from existing split file: /content/nnUNET_SELs/nnUNet_SEL/nnUNet_preprocessed/Dataset101_SEL/splits_final.json
2024-05-04 09:25:02.327195: The split file contains 5 splits.
2024-05-04 09:25:02.327255

In [ ]:
!nnUNetv2_train Dataset101_SEL 3d_fullres 4 -tr nnUNetTrainer_10epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-05-04 09:52:50.068980: do_dummy_2d_data_aug: False
2024-05-04 09:52:50.069880: Using splits from existing split file: /content/nnUNET_SELs/nnUNet_SEL/nnUNet_preprocessed/Dataset101_SEL/splits_final.json
2024-05-04 09:52:50.070134: The split file contains 5 splits.
2024-05-04 09:52:50.070198

In [ ]:
!nnUNetv2_predict -d Dataset101_SEL -i /content/nnUNet_SEL/nnUNet_raw/Dataset101_SEL/imagesTs -o /content/nnUNet_SEL/nnUNet_results/Dataset101_SEL/inference -f  0 1 2 3 4 -tr nnUNetTrainer_10epochs -c 3d_fullres -p nnUNetPlans


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 20 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 20 cases that I would like to predict
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Predicting 131:
perform_everything_on_device: True
100% 8/8 [00:06<00:00,  1.29it/s]
100% 8/8 [00:04<00:00,  2.00it/s]
100% 8/8 [00:04<00:00,  1.99it/s]
100% 8/8 [00:04<00:00,  1.99it/s]
100% 8/8 [00:04<

In [ ]:
!nnUNetv2_apply_postprocessing -i /content/nnUNet/nnUNet_results/Dataset101/inference -o /content/nnUNet/nnUNet_results/Dataset101/postprocessing -pp_pkl_file /content/nnUNet/nnUNet_results/Dataset101/nnUNetTrainer_10epochs__nnUNetPlans__3d_fullres/crossval_results_folds_0_1_2_3_4/postprocessing.pkl -np 8 -plans_json /content/nnUNet/nnUNet_results/Dataset101/nnUNetTrainer_10epochs__nnUNetPlans__3d_fullres/crossval_results_folds_0_1_2_3_4/plans.json